In [ ]:
# Jason Hemedinger
# Argonne National Laboratory


In [ ]:
import pyart, boto3, tempfile, os, shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from botocore.handlers import disable_signing
from tint import Cell_tracks
from tint import animate as tint_animate
from tint.visualization import embed_mp4_as_gif
from glob import glob
from datetime import datetime
from pylab import *

In [ ]:
def get_current_scan(station_date, key_index):
    '''
    Function will pull the latest radar scan from any radar site using 
    Amazon S3.
    ----------
    station_date= 'year/month/day/station'
    year = YYYY e.g. 1776
    month = mm e.g. 07
    day = dd e.g. 04
    site = xxxx e.g. kdix
    '''
    bucket = 'noaa-nexrad-level2'
    s3 = boto3.resource('s3')
    s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    aws_radar = s3.Bucket(bucket)
    
    for obj in aws_radar.objects.filter(Prefix= station_date):
        '{0}:{1}'.format(aws_radar.name, obj.key)
    my_list_of_keys = [this_object.key for this_object in aws_radar.objects.filter(Prefix= station_date)]
    keys = my_list_of_keys[key_index:]
    print(my_list_of_keys[key_index:])
    
    
    return aws_radar, keys

In [ ]:
#setting the radar site and which keys to pull.
aws_radar, keys = get_current_scan('2018/07/2/KLOT', key_index=-15)

In [ ]:
#animating using matplotlib and pyart
def animate(nframe):
    plt.clf()
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(keys[nframe], localfile.name)
    radar = pyart.io.read(localfile.name)
    display = pyart.graph.RadarMapDisplay(radar)
    # Delete radar after use to save memory.
    del radar
    display.plot_ppi_map('reflectivity', sweep=0, resolution='l',
                         vmin=-8, vmax=64, mask_outside=False, 
                         fig=fig, width=350000, height=350000, 
                        cmap = pyart.graph.cm.LangRainbow12 )
    display.basemap.drawcounties()
    display.plot_point(-87.981810, 41.713969 , label_text='ANL', symbol='ko')
#    display.plot_point(-87.04361, 41.46361, label_text='VU', color='k')
fig = plt.figure(figsize=(10, 8))
anim_klot = animation.FuncAnimation(fig, animate,
                                    frames=len(keys))
anim_klot.save('reflectivity_animation.gif', 
               writer='imagemagick', fps=2)
plt.show()
plt.close()

In [ ]:
out_path_dir = '/home/jhemedinger/suli_projects/chicago-nowcast/events/2018_06_26/testing'
#out_path_dir = '/home/jhemedinger/suli_projects/chicago-nowcast/events/2018_06_26/20180626-22Z'
#out_path_dir2 = '/home/jhemedinger/suli_projects/chicago-nowcast/events/2018_06_26/20180626-23Z'

In [ ]:
#turing the data into grid data and saving it to a folder
def get_grid(aws_radar, key):
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(key, localfile.name)
    radar = pyart.io.read(localfile.name)
    grid = pyart.map.grid_from_radars(
            radar, grid_shape=(31, 401, 401),
            grid_limits=((0, 15000), (-200000, 200000), (-200000, 200000)),
            fields=['reflectivity'], gridding_algo='map_gates_to_grid',
            h_factor=0., nb=0.6, bsp=1., min_radius=200.)
    return grid

#out_path_dir = '/home/jhemedinger/suli_projects/chicago-nowcast/events

for num,key in enumerate(keys):
    print('saving grid', num)
    grid = get_grid(aws_radar, key)
    name = os.path.join(out_path_dir, 'grid_' + str(num).zfill(3) + '.nc')
    pyart.io.write_grid(name, grid)
    del grid

In [ ]:
#files_2 = [os.path.join(out_dir, fn) for fn in os.listdir(out_dir)]
files = glob(out_path_dir + '/grid_*')
files.sort()

In [ ]:
grid_gen = (pyart.io.read_grid(f) for f in files)
#grid_list = [pyart.io.read_grid(f) for f in files]

In [ ]:
tracks_obj = Cell_tracks()
tracks_obj.params

In [ ]:
tracks_obj.params['FIELD_THRESH']=35
#tracks_obj.params['min_size']=16

In [ ]:
tracks_obj.get_tracks(grid_gen)

In [ ]:
tracks_obj.tracks

In [ ]:
if os.path.exists(out_path_dir + '/test_animation.mp4'):
    print(out_path_dir + '/test_animation.mp4'
          + ' already exists, removing file')
    os.remove(out_path_dir + '/test_animation.mp4')

In [ ]:
grid_gen = (pyart.io.read_grid(f) for f in files)
tint_animate(tracks_obj, grid_gen, os.path.join(out_path_dir, 'test_animation'), tracers=True)

In [ ]:
embed_mp4_as_gif(out_path_dir + '/test_animation.mp4')

In [ ]:
cells = tracks_obj.tracks.groupby(level='uid')
for uid in cells:
    print(uid)

In [ ]:
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:]

In [ ]:
df_161 = pd.DataFrame(tracks_obj.tracks.xs('161', level='uid'))
lons, lats = np.array(df_161['lon']), np.array(df_161['lat'])
time = np.array(pd.to_datetime(df_161['time']))

In [ ]:
fit = polyfit(lons,lats,1)
fit_fn = poly1d(fit)

fig = plt.figure(figsize=(10,8))
plt.plot(lons, fit_fn(lons), '--b')
plt.plot(lons, lats, color='k', marker='o')
#plt.plot(-87.981810, 41.713969, color='r', marker='*')
plt.xlabel('LONGITUDE')
plt.ylabel('LATITUDE')
plt.show()
plt.close()

In [ ]:
t = (time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
x, y = lats, lons


fit_lat = polyfit(t,x,1)
fit_lon = polyfit(t,y,1)
fit_fn_lon = poly1d(fit_lon)
fit_fn_lat = poly1d(fit_lat)

fig = plt.figure(figsize=(10,8))
plt.plot(time, x, 'ro', time, fit_fn_lat(t), '--k')
plt.xlabel('TIME (UTC)')
plt.ylabel('LATITUDE')
plt.show()
plt.close()

fig = plt.figure(figsize=(10,8))
plt.plot(time, y, 'bo', time, fit_fn_lon(t), '--k')
plt.xlabel('TIME (UTC)')
plt.ylabel('LONGITUDE')
plt.show()
plt.close()

In [ ]:
def lats_lons(minimum, maximum, interval):
    '''
    Will predict lat/lon for a given time interval.
    Returns time, lat, and lon
    beginning: beginning of the time interval
    end: end of interval
    interval: time interval in minutes
    
    Ex: lats_lons(10, 70, 10) will find the lat/lon 
    for the next hour every 10 minutes.
    '''
    minimum = minimum
    maximum = maximum
    interval = interval
    arr = np.arange(minimum, maximum, interval) 
    my_time = []
    for i in arr:
        my_time.append(time[-1] + np.timedelta64(str(i), 'm'))
    my_new_time = np.array(my_time)
    nts = ((my_new_time - np.datetime64('1970-01-01T00:00:00Z')) 
           / np.timedelta64(1, 's'))
    my_new_lat = fit_fn_lat(nts)
    my_new_lon = fit_fn_lon(nts)
    print(my_new_lon)
    print(my_new_lat)
    return my_new_time, my_new_lat, my_new_lon

In [ ]:
my_new_time, my_new_lat, my_new_lon = lats_lons(10, 110,10)

In [ ]:
#animating using matplotlib and pyart
def animate(nframe):
    plt.clf()
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(keys[nframe], localfile.name)
    radar = pyart.io.read(localfile.name)
    display = pyart.graph.RadarMapDisplay(radar)
    # Delete radar after use to save memory.
    del radar
    display.plot_ppi_map('reflectivity', sweep=0, resolution='l',
                         vmin=-8, vmax=64, mask_outside=True, 
                         fig=fig, width=200000, height=200000,
                         cmap=pyart.graph.cm.LangRainbow12)
    display.basemap.drawcounties()
    display.plot_line_geo(my_new_lon, my_new_lat, '--r')
    display.plot_line_geo(lons[:nframe], lats[:nframe], '-k')
    display.plot_point(-87.981810, 41.713969 , label_text='ANL', symbol='ko')
fig = plt.figure(figsize=(10, 8))

anim_klot = animation.FuncAnimation(fig, animate, 
                                    frames=len(keys))
anim_klot.save(out_path_dir + '/reflectivity_animation_bleh.gif', 
               writer='imagemagick', fps=1)
plt.show()
plt.close()